### Step 1: Load modules and functions

In [1]:
import xarray as xr
import h5py
from brainio.assemblies import NeuronRecordingAssembly
from pynwb import NWBHDF5IO, NWBFile
import glob, os, yaml
import pytz  # This is required to handle timezone conversions
from datetime import datetime
from uuid import uuid4
import numpy as np
import scipy.io
import os, glob, json
import pandas as pd
from pynwb.file import Subject
import logging, sys

cwd = os.getcwd()
sys.path.append(os.path.dirname(cwd))
root_dir        = '/braintree/home/aliya277/inventory_new'

def read_names(filename):
    assignment  = filename.split('.')[0].split('-')[1]
    number      = filename.split('.')[0].split('-')[2]
    return np.asarray([assignment, number])

def create_combined_nwb(config, path, experiment_name):

    desired_timezone = pytz.timezone('US/Eastern')

    ################ CREATE NWB FILE WITH METADATA ################################
    ###############################################################################
    nwbfile = NWBFile(
        session_description     = 'Integrated NWB file for the experiment, featuring PSTH data verified for quality from each session.',
        identifier              = str(uuid4()),
        session_start_time      = desired_timezone.localize(config['metadata']['session_start_time']),
        file_create_date        = datetime.now(desired_timezone), #desired_timezone.localize(config['metadata']['file_create_date']),
        experimenter            = config['general']['lab_info']['experimenter'],
        experiment_description  = config['general']['experiment_info']['experiment_description'],
        session_id              = experiment_name, #config['session_info']['session_id'],
        lab                     = config['general']['lab_info']['lab'],                     
        institution             = config['general']['lab_info']['institution'],                                    
        keywords                = config['general']['experiment_info']['keywords'],
        surgery                 = config['general']['experiment_info']['surgery']
    )

    ################ CREATE SUBJECT ################################################
    ################################################################################
    nwbfile.subject = Subject(
        subject_id  = config['subject']['subject_id'],
        date_of_birth= config['subject']['date_of_birth'],
        species     = config['subject']['species'],
        sex         = config['subject']['sex'],
        description = config['subject']['description'],
    )

    ################ CREATE HARDWARE LINKS #########################################
    ################################################################################
    nwbfile.create_device(
        name        = config['hardware']['system_name'], 
        description = config['hardware']['system_description'], 
        manufacturer= config['hardware']['system_manuf']
    )

    nwbfile.create_device(
        name        = config['hardware']['adapter_manuf'], 
        description = config['hardware']['adapter_description'], 
        manufacturer= config['hardware']['adapter_manuf']
    )

    nwbfile.create_device(
        name        = config['hardware']['monitor_name'], 
        description = config['hardware']['monitor_description'], 
        manufacturer= config['hardware']['monitor_manuf']
    )

    nwbfile.create_device(
        name        = config['hardware']['photodiode_name'], 
        description = config['hardware']['photodiode_description'], 
        manufacturer= config['hardware']['photodiode_manuf']
    )
    
    nwbfile.create_device(
        name        = 'Software Used', 
        description = str(['Mworks Client: '+config['software']['mwclient_version'],\
                        'Mworks Server: '+config['software']['mwserver_version'],\
                        'OS: '+config['software']['OS'],\
                        'Intan :'+config['software']['intan_version']])
    )

    ################ CREATE ELECTRODE LINKS ########################################
    ################################################################################
    electrodes = nwbfile.create_device(
        name        = config['hardware']['electrode_name'], 
        description = config['hardware']['electrode_description'], 
        manufacturer= config['hardware']['electrode_manuf']
    )

    all_files = sorted(os.listdir(os.path.join(path, 'SpikeTimes')))
    
    name_accumulator = []
    for file in all_files:
        name_accumulator.append(read_names(file))
    names = np.vstack(name_accumulator)

    nwbfile.add_electrode_column(name="label", description="label of electrode")
    groups, count_groups = np.unique(names[:,0], return_counts =True)
    ids                  = names[:,1]
    counter              = 0
    # create ElectrodeGroups A, B, C, ..
    for group, count_group in zip(groups, count_groups):
        if len(groups) == 6:
            electrode_description = "Serialnumber: {}. Adapter Version: {}".format(config['array_info']['array_{}'.format(group)]['serialnumber'],\
                            config['array_info']['array_{}'.format(group)]['adapterversion']),
        else: 
            electrode_description = "Serialnumber: {}".format(config['array_info']['array_{}'.format(group)]['serialnumber']),
                
        
        electrode_group = nwbfile.create_electrode_group(
            name        = "group_{}".format(group),
            description = electrode_description[0],
            device      = electrodes,
            location    = 'hemisphere, region, subregion: '+str([config['array_info']['array_{}'.format(group)]['hemisphere'],\
                                config['array_info']['array_{}'.format(group)]['region'],
                                config['array_info']['array_{}'.format(group)]['subregion']]),
            position    = config['array_info']['array_{}'.format(group)]['position']
        )

        # create Electrodes 001, 002, ..., 032 in ElectrodeGroups per channel
        for ichannel in range(count_group):
            nwbfile.add_electrode(
                group       = electrode_group,
                label       = ids[counter],
                location    = 'row, col, elec'+str(json.loads(config['array_info']['intan_electrode_labeling_[row,col,id]'])[counter])
            )
            counter += 1     


    return nwbfile

df = pd.read_excel( os.path.dirname(cwd)+'/pico_inventory.xlsx' , sheet_name='Sheet2')
df['combined_nwb'] = ''
df['train_test_nwb'] = ''


### Step 2: Create per experiment nwb file with good channels only

In [2]:
def experiment_processed(df, exp_path):
    imageset = os.path.basename(exp_path).split('.')[0].split('_')[1:]
    
    if len(imageset) == 1: imageset = imageset[0]
    elif len(imageset) > 1: imageset = '_'.join(imageset)
    mask = df['ImageSet'] == imageset
    index = df.index[mask].tolist()[0]
    if df.iloc[index]['proc_nwb'].startswith("P-Values added."): return True
    else: 
        print("    ", df.iloc[index]['proc_nwb'])
        return False

def update_sheet(df, exp_path, text, which_nwb):
    imageset = os.path.basename(exp_path).split('.')[0].split('_')[1:]
    if len(imageset) == 1: imageset = imageset[0]
    elif len(imageset) > 1: imageset = '_'.join(imageset)
    mask = df['ImageSet'] == imageset
    index = df.index[mask].tolist()[0]
    if which_nwb == 'combined':
        df.at[index, 'combined_nwb'] = text
    if which_nwb == 'train_test':
        df.at[index, 'train_test_nwb'] = text

In [9]:
def find_norm_with_date(realdate):
    all_paths = []
    normalizer_file_paths = glob.glob(os.path.join(root_dir, '[norm]*', '*', '*', '[!h5]*'))
    for norm_file_path in normalizer_file_paths:
        date, time = os.path.basename(norm_file_path).split('.')[-2].split('_')
        if date == realdate: 
            all_paths.append(norm_file_path)

    if len(all_paths) == 0: return None   
    
    return all_paths

def normalized_psth_across_sessions(psth, realdate):

    norm_paths = find_norm_with_date(realdate)
    for norm_path in norm_paths:

        if norm_path != None:
            norm_nwb_file_path = glob.glob(os.path.join(norm_path, '*[nwb]'))[0]
            
            print(f"Within: using normalizer file {os.path.basename(norm_nwb_file_path)}")
            io = NWBHDF5IO(norm_nwb_file_path, "r") 
            norm_nwbfile = io.read()
            try: 
                normalizer_psth = norm_nwbfile.scratch['psth'][:]
                normalizer_meta = norm_nwbfile.scratch['psth meta'][:]
                io.close()
            except Exception as error: print(error)
            
            if normalizer_psth.shape[-1] != psth.shape[-1]: continue 
            
            assert len(normalizer_psth.shape) == 4 , 'Normalizer PSTH has wrong shape.' # num_images x num_repetitions x num_timebins x num_channels
            
            timebase = np.arange(int(normalizer_meta[0]), int(normalizer_meta[1]), int(normalizer_meta[2]))
            t_cols = np.where((timebase >= 70) & (timebase < 170))[0]

            if normalizer_psth.shape[0] == 86: # 85 normalizers images + 1 gray image
                images_ids = np.arange(0,86)
                images_no_grey = np.where(images_ids != 26)[0]
                normalizer_p_no_grey = normalizer_psth[images_no_grey,:,:,:] # Select all images except grey (#26),

                n_p = np.nanmean(normalizer_p_no_grey[:, :, t_cols, :], 2) # then mean 70-170 time bins

                assert n_p.shape[0] == 85, 'PSTH has wrong no. stimuli'
                n_p = n_p.reshape(-1, normalizer_p_no_grey.shape[-1])  # Reshape so that first two axes collapse into one
            else:
                images_ids = np.arange(0,normalizer_psth.shape[0])
                normalizer_p_no_grey = normalizer_psth[images_ids,:,:,:] 
                n_p = np.nanmean(normalizer_p_no_grey[:, :, t_cols, :], 2) # then mean 70-170 time bins
                n_p = n_p.reshape(-1, normalizer_p_no_grey.shape[-1])  # Reshape so that first two axes collapse into one


            mean_response_normalizer = np.nanmean(n_p, 0)  # Mean across images x reps
            std_response_normalizer = np.nanstd(n_p, 0)  # Std across images x reps

            p = np.subtract(psth, mean_response_normalizer[np.newaxis, np.newaxis, np.newaxis, :])
            p = np.divide(psth, std_response_normalizer[np.newaxis, np.newaxis, np.newaxis, :],
                            where=std_response_normalizer!=0)
            
            return p

        else: 
            print(f'No normalizer found for day {realdate}.')
            return [None]
 

In [12]:
experiment_file_paths = glob.glob(os.path.join(root_dir, '[exp]*', '*'))
for experiment_path in experiment_file_paths: 

    if os.path.basename(experiment_path)=='VideoStimulusSet': continue 
    
    days    = glob.glob(os.path.join(experiment_path, '*[!npy][!nwb]'))
    n_days  = len(days)
    n_sessions = 0
    for day in days :
        n_sessions += len(glob.glob(os.path.join(experiment_path, day, '*',  '*[nwb]')))

    # if not os.path.basename(experiment_path)=='exp_shapegen_static.sub_pico': continue 

    print('________________________________________________________________________________')
    print(f'{os.path.basename(experiment_path)} has {n_days} days and {n_sessions} sessions')
    
    # ------------------------------------------------------------------------------ 
    # Skip files, which have no nwb files or if combined nwb file already exists.
    # ------------------------------------------------------------------------------ 
    if n_sessions == 0: continue
    combined_exists = False
    train_exists    = False
    test_exists     = False

    if os.path.isfile(os.path.join(experiment_path, f"{os.path.basename(experiment_path)}_combined.nwb")):
        print(f'Combined nwb file exists for {os.path.basename(experiment_path)}')
        update_sheet(df, experiment_path, 'Combined nwb file exists.', 'combined')
        combined_exists = True

    if os.path.isfile(os.path.join(experiment_path, f"{os.path.basename(experiment_path)}_combined_train.nwb")):
        print(f'Combined train nwb file exists for {os.path.basename(experiment_path)}')
        update_sheet(df, experiment_path, 'Combined train nwb file exists.', 'train_test')
        train_exists    = True
    if os.path.isfile(os.path.join(experiment_path, f"{os.path.basename(experiment_path)}_combined_test.nwb")):
        print(f'Combined test nwb file exists for {os.path.basename(experiment_path)}')
        update_sheet(df, experiment_path, 'Combined test nwb file exists.' , 'train_test')
        test_exists     = True
        
    if combined_exists and train_exists and test_exists: continue 

    # ------------------------------------------------------------------------------ 
    # Create combined nwb files, if all previous steps are taken.
    # ------------------------------------------------------------------------------ 
    if experiment_processed(df, experiment_path) == True: 
        first_day = days[0].split('.')[-1]

        with open(os.path.join(glob.glob(os.path.join(experiment_path, days[0], '*'))[0],f"config_nwb.yaml") , "r") as f:
                first_rec_config = yaml.load(f, Loader = yaml.FullLoader)
                if combined_exists == False: combined_nwb = create_combined_nwb(first_rec_config, glob.glob(os.path.join(experiment_path, days[0], '*'))[0], os.path.basename(days[0]).split('.')[0])
                combined_nwb_train  = create_combined_nwb(first_rec_config, glob.glob(os.path.join(experiment_path, days[0], '*'))[0], os.path.basename(days[0]).split('.')[0])
                combined_nwb_test   = create_combined_nwb(first_rec_config, glob.glob(os.path.join(experiment_path, days[0], '*'))[0], os.path.basename(days[0]).split('.')[0])
            
        combined_good_channel_ids = []
        combined_date_times = []
        combined_train_IDs  = []
        combined_test_IDs   = []
        
        # ------------------------------------------------------------------------------ 
        # Load all the recording nwb files and combine them to an experiment nwb file.
        # ------------------------------------------------------------------------------ 

        pass_saving       = False
        all_good_channels = []
        all_norm_psth     = []
        for day, i_day in zip(days, range(n_days)) :
            print(f'Loading files from day {i_day+1}/{n_days}')
            exp_nwb_paths = (glob.glob(os.path.join(experiment_path, day, '*',  '*[nwb]')))
            
            for exp_nwb_path in exp_nwb_paths:
                
                # ------------------------------------------------------------------------------ 
                # Load recording nwb file.
                # ------------------------------------------------------------------------------ 
                try:
                    io = NWBHDF5IO(exp_nwb_path, "r") 
                    exp_nwbfile = io.read()
                except:
                    print(f"Cannot open nwb. file {os.path.basename(exp_nwb_path)}")
                    io.close()
                    update_sheet(df, experiment_path, f"Cannot open nwb. file {os.path.basename(exp_nwb_path)}", 'combined')
                    pass_saving = True
                    continue 
                    
                try: 
                    psth = exp_nwbfile.scratch['psth'][:]
                    psth_meta = exp_nwbfile.scratch['psth meta'][:]

                    # ------------------------------------------------------------------------------ 
                    # Remove all the not bad channels.
                    # ------------------------------------------------------------------------------ 
                    p_values = exp_nwbfile.scratch['RecordingQualityArray'][:]
                    good_channel_ids = p_values<0.05
                    combined_good_channel_ids.append(good_channel_ids)
                    combined_date_times.append(os.path.basename(exp_nwb_path).split('.')[-3])
                    good_psth = psth[:,:,:,good_channel_ids]
                    if np.isnan(good_psth).sum() != 0: good_psth = good_psth[:,:-1,:,:] # remove last rep of nans 


                    # ------------------------------------------------------------------------------ 
                    # If n_sessions > 1, append all p-values and normalized psth into two lists.
                    # ------------------------------------------------------------------------------ 
                    if n_sessions > 1:
                        current_day = day.split('.')[-1]
                        if np.isnan(psth).sum() != 0: psth = psth[:,:-1,:,:] # remove last rep of nans 
                        norm_psth = normalized_psth_across_sessions(psth, current_day)
                        print(f"Normalized and Original PSTH {norm_psth.shape} {psth.shape}")
                        all_norm_psth.append(norm_psth)

                    # display(exp_nwbfile)

                    # ------------------------------------------------------------------------------ 
                    # Define Train and Test set.
                    # ------------------------------------------------------------------------------ 
                    n_stimuli = psth.shape[0]
                    n_test    = int(n_stimuli*0.15)
                    train_ids = np.linspace(0, n_stimuli-n_test-1, n_stimuli-n_test, dtype=int)
                    test_ids  = np.linspace(n_stimuli-n_test, n_stimuli-1, n_test, dtype=int)
                    combined_train_IDs.append(train_ids)
                    combined_test_IDs.append(test_ids)

                    if train_ids[-1] != test_ids[0]-1: 
                        raise ValueError("Error in Test and Train IDs!") 
                    

                    # ------------------------------------------------------------------------------ 
                    # Add to combined nwb files. One containing all the data, one test and one train.
                    # ------------------------------------------------------------------------------ 
                    
                    if combined_exists == False:
                        combined_nwb.add_scratch(
                            good_psth,
                            name=f"QualityCheckedPSTH_{os.path.basename(exp_nwb_path).split('.')[-3]}",
                            description=f"PSTH array with dimensions corresponding to \
                                [stimuli x repetitions x time bins x good quality channels], \
                                where 'quality channels' are those with p-values less \
                                than 0.05, indicating high signal quality. The 'stimulus ID' \
                                corresponds to the index in the 'stimuli' dimension. The PSTH meta \
                                are the following for [start_time_ms, stop_time_ms, tb_ms]: {psth_meta}"
                            )
                    
                    if train_exists == False:
                        combined_nwb_train.add_scratch(
                            good_psth[train_ids,:,:,:],
                            name=f"QualityCheckedPSTH_{os.path.basename(exp_nwb_path).split('.')[-3]}",
                            description="PSTH array with dimensions corresponding to \
                                [stimuli x repetitions x time bins x good quality channels], \
                                where 'quality channels' are those with p-values less \
                                than 0.05, indicating high signal quality. This train-set \
                                contains the data corresponding to the first 85% of the stimuli.\
                                The 'stimulus ID' matches the index in the 'stimuli' dimension. \
                                A corresponding index-to-stimulus ID mapping is available in TrainStimuliIDs. The PSTH meta \
                                are the following for [start_time_ms, stop_time_ms, tb_ms]: {psth_meta}"
                            )
                        
                    if test_exists == False:
                        combined_nwb_test.add_scratch(
                            good_psth[test_ids,:,:,:],
                            name=f"QualityCheckedPSTH_{os.path.basename(exp_nwb_path).split('.')[-3]}",
                            description=f"PSTH array with dimensions corresponding to \
                                [stimuli x repetitions x time bins x good quality channels], \
                                where 'quality channels' are those with p-values less \
                                than 0.05, indicating high signal quality. This test-set \
                                contains the data corresponding to the last 15% of the stimuli. \
                                The 'stimulus ID' does NOT match the index in the 'stimuli' dimension. \
                                A corresponding index-to-stimulus ID mapping is available in TestStimuliIDs. The PSTH meta \
                                are the following for [start_time_ms, stop_time_ms, tb_ms]: {psth_meta}"
                            )
                    
                except Exception as error:
                    print("An error occurred:", error) 
                    io.close()
                    pass_saving = True
                    update_sheet(df, experiment_path, error, 'combined')
                    continue 

                io.close()
        
        #print(all_good_channels)
        # Use numpy.logical_and to find common True indices across all arrays

        # ------------------------------------------------------------------------------ 
        # If n_sessions > 1, combine all sessions using the common good channels.
        # ------------------------------------------------------------------------------
        if n_sessions > 1:
            combined_good_psth = []
            combined_good_psth_train = []
            combined_good_psth_test  = []
            common_true_indices = np.logical_and.reduce(combined_good_channel_ids)
            for norm_psth, i, train_ids, test_ids in zip(all_norm_psth, range(len(all_norm_psth)), combined_train_IDs, combined_test_IDs):
                if i == 0:
                    combined_good_psth       = norm_psth[:,:,:,common_true_indices]
                    combined_good_psth_train = combined_good_psth[list(train_ids),:,:,:]
                    combined_good_psth_test  = combined_good_psth[list(test_ids),:,:,:]
                else:
                    combined_temp       = norm_psth[:,:,:,common_true_indices]
                    combined_good_psth  = np.hstack((combined_good_psth, combined_temp))
                    combined_good_psth_train = np.hstack((combined_good_psth_train, combined_temp[list(train_ids),:,:,:]))
                    combined_good_psth_test  = np.hstack((combined_good_psth_test,  combined_temp[list(test_ids),:,:,:]))
        
        # print(f"Combined PSTH: {combined_good_psth.shape}")
        # print(f"Combined PSTH train: {combined_good_psth_train.shape}")
        # print(f"Combined PSTH test : {combined_good_psth_test.shape}")


        if pass_saving == True: continue
        
        # ------------------------------------------------------------------------------ 
        # Add masks and stimIDs to to combined nwb files.
        # ------------------------------------------------------------------------------ 

        if combined_exists==False: 
            combined_nwb.add_scratch(
            combined_good_channel_ids,
            name=f"QualityElectrodesMasks",
            description=f"List of boolean arrays, indexed by recording date & time, each of num-electrode length, marking 'good quality channels' as True for p-value < 0.05. Dates & times covered:{combined_date_times}"
            )
            if n_sessions > 1:
                combined_nwb.add_scratch(
                    combined_good_psth,
                    name=f"CombinedQualityCheckedPSTHs",
                    description=f"Array of shape [stimuli x all repetitions x time bins x logical 'and' of good quality channels], \
                        containing all PSTHs stacked in one matrix. All repetitions are stacked, and the good quality channels \
                        of each recording are combined using a logical 'and' operation, i.e., \
                        common_true_indices = np.logical_and.reduce(QualityElectrodesMasks). \
                        The resulting common_true_indices is used to mask the channel dimensions. \
                        The PSTH meta are the following for [start_time_ms, stop_time_ms, tb_ms]: {[0,300,10]}\
                        This array contains the cleaned recordings from the following Dates & Times: \
                        {combined_date_times}"
            )

        if train_exists == False: 
            combined_nwb_train.add_scratch(
                combined_good_channel_ids,
                name=f"QualityElectrodesMasks",
                description=f"List of boolean arrays, indexed by recording date & time, each of num-electrode length, marking 'good quality channels' as True for p-value < 0.05. Dates & times covered:{combined_date_times}"
                )
            combined_nwb_train.add_scratch(
                combined_train_IDs,
                name=f"TrainStimuliIDs",
                description=f"List of arrays, indexed by recording date and time, contains stimulus IDs that directly correspond to the stimulus indexes in QualityCheckedPSTHs. \
                Each entry in an array aligns with the respective stimulus ID in the PSTH; for example, the first entry in the array corresponds to the stimulus ID of the first entry in the PSTH.\
                The corresponding stimulus for each ID is located in the respective stimulus set. Dates & times covered: {combined_date_times}"
                )
            if n_sessions > 1:
                combined_nwb_train.add_scratch(
                    combined_good_psth_train,
                    name=f"CombinedQualityCheckedPSTHs",
                    description=f"Array of shape [stimuli x all repetitions x time bins x logical 'and' of good quality channels], \
                        containing all PSTHs stacked in one matrix. All repetitions are stacked, and the good quality channels \
                        of each recording are combined using a logical 'and' operation, i.e., \
                        common_true_indices = np.logical_and.reduce(QualityElectrodesMasks). \
                        The resulting common_true_indices is used to mask the channel dimensions. \
                        The TrainStimuliIDs are used to mask the simulus dimension.\
                        The PSTH meta are the following for [start_time_ms, stop_time_ms, tb_ms]: {[0,300,10]}\
                        This array contains the cleaned recordings from the following Dates & Times: \
                        {combined_date_times}"
                )

        if test_exists==False:
            combined_nwb_test.add_scratch(
                combined_good_channel_ids,
                name=f"QualityElectrodesMasks",
                description=f"List of boolean arrays, indexed by recording date & time, each of num-electrode length, marking 'good quality channels' as True for p-value < 0.05. Dates & times covered:{combined_date_times}"
                )
            combined_nwb_test.add_scratch(
                combined_test_IDs,
                name=f"TestStimuliIDs",
                description=f"List of arrays, indexed by recording date and time, contains stimulus IDs that directly correspond to the stimulus indexes in QualityCheckedPSTHs. \
                Each entry in an array aligns with the respective stimulus ID in the PSTH; for example, the first entry in the array corresponds to the stimulus ID of the first entry in the PSTH.\
                The corresponding stimulus for each ID is located in the respective stimulus set. Dates & times covered: {combined_date_times}"
                )
            if n_sessions > 1:
                combined_nwb_test.add_scratch(
                    combined_good_psth_test,
                    name=f"CombinedQualityCheckedPSTHs",
                    description=f"Array of shape [stimuli x all repetitions x time bins x logical 'and' of good quality channels], \
                        containing all PSTHs stacked in one matrix. All repetitions are stacked, and the good quality channels \
                        of each recording are combined using a logical 'and' operation, i.e., \
                        common_true_indices = np.logical_and.reduce(QualityElectrodesMasks). \
                        The resulting common_true_indices is used to mask the channel dimensions. \
                        The TestStimuliIDs are used to mask the simulus dimension.\
                        The PSTH meta are the following for [start_time_ms, stop_time_ms, tb_ms]: {[0,300,10]}\
                        This array contains the cleaned recordings from the following Dates & Times: \
                        {combined_date_times}"
                    )
    
        
        update_sheet(df, experiment_path, 'Combined nwb file exists.', 'combined')
        update_sheet(df, experiment_path, 'Combined train nwb file exists.', 'train_test')
        update_sheet(df, experiment_path, 'Combined test nwb file exists.' , 'train_test')

        # ------------------------------------------------------------------------------ 
        # Save experiment nwb file.
        # ------------------------------------------------------------------------------ 
        print('Saving combined NWB Files.')
        if combined_exists==False: 
            io = NWBHDF5IO(os.path.join(experiment_path, f"{os.path.basename(experiment_path)}_combined.nwb"), "w") 
            io.write(combined_nwb)
            io.close()
            print("Combined file saved.")
        
        if train_exists == False: 
            io = NWBHDF5IO(os.path.join(experiment_path, f"{os.path.basename(experiment_path)}_combined_train.nwb"), "w") 
            io.write(combined_nwb_train)
            io.close()
            print("Train file saved.")
        
        if test_exists==False:
            io = NWBHDF5IO(os.path.join(experiment_path, f"{os.path.basename(experiment_path)}_combined_test.nwb"), "w") 
            io.write(combined_nwb_test)
            # display(combined_nwb_test)
            io.close()
            print("Test file saved.")
            
    
# Update Sheet 2
xls = pd.ExcelFile(f'{os.path.dirname(cwd)}/pico_inventory.xlsx')
sheets = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}

sheets['Sheet2'] = df  

with pd.ExcelWriter(f'{os.path.dirname(cwd)}/pico_inventory.xlsx', engine='openpyxl', mode='w') as writer:
    for sheet_name, sheet_df in sheets.items():
        sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)         

________________________________________________________________________________
exp_muri1320.sub_pico has 11 days and 12 sessions
     Across: using normalizer file norm_FOSS.sub_pico.20220615_113442.proc.nwb and norm_FOSS.sub_pico.20220706_141433.proc.nwb 
Number of channels do not match. 288 != 192 
Across: using normalizer file norm_FOSS.sub_pico.20220615_113442.proc.nwb and norm_FOSS.sub_pico.20220706_142235.proc.nwb 
'psth'
No psth available for normalizers ('20220615', '20220706').

________________________________________________________________________________
exp_ko_context_size.sub_pico has 1 days and 1 sessions
Loading files from day 1/1


Saving combined NWB Files.
Combined file saved.
Train file saved.
Test file saved.
________________________________________________________________________________
exp_robustness_guy_d1_v40.sub_pico has 1 days and 3 sessions
Loading files from day 1/1
Within: using normalizer file norm_FOSS.sub_pico.20230928_101016.proc.nwb


/tmp/ipykernel_155137/3704893679.py:30: RuntimeWarning: Mean of empty slice
  n_p = np.nanmean(normalizer_p_no_grey[:, :, t_cols, :], 2) # then mean 70-170 time bins


Normalized and Original PSTH (330, 50, 30, 192) (330, 50, 30, 192)
Within: using normalizer file norm_FOSS.sub_pico.20230928_101016.proc.nwb
Normalized and Original PSTH (330, 4, 30, 192) (330, 4, 30, 192)
Within: using normalizer file norm_FOSS.sub_pico.20230928_101016.proc.nwb
Normalized and Original PSTH (330, 11, 30, 192) (330, 11, 30, 192)
Saving combined NWB Files.
Combined file saved.
Train file saved.
Test file saved.
________________________________________________________________________________
exp_muri1320-2023-v1.sub_pico has 4 days and 8 sessions
Loading files from day 1/4
Within: using normalizer file norm_FOSS.sub_pico.20230127_160227.proc.nwb
Normalized and Original PSTH (1320, 4, 30, 192) (1320, 4, 30, 192)
Within: using normalizer file norm_FOSS.sub_pico.20230127_160227.proc.nwb
Normalized and Original PSTH (1320, 5, 30, 192) (1320, 5, 30, 192)
Loading files from day 2/4
Within: using normalizer file norm_FOSS.sub_pico.20230130_140402.proc.nwb
Normalized and Original

/tmp/ipykernel_155137/3704893679.py:37: RuntimeWarning: Mean of empty slice
  n_p = np.nanmean(normalizer_p_no_grey[:, :, t_cols, :], 2) # then mean 70-170 time bins


Normalized and Original PSTH (300, 8, 200, 192) (300, 8, 200, 192)
Loading files from day 2/4
Within: using normalizer file norm_HVM.sub_pico.20230502_145301.proc.nwb
Normalized and Original PSTH (300, 8, 200, 192) (300, 8, 200, 192)
Loading files from day 3/4
An error occurred: 'psth'
Loading files from day 4/4
Within: using normalizer file norm_HVM.sub_pico.20230504_114437.proc.nwb
Normalized and Original PSTH (300, 8, 200, 192) (300, 8, 200, 192)
________________________________________________________________________________
exp_IAPS.sub_pico has 2 days and 2 sessions
Loading files from day 1/2
Within: using normalizer file norm_FOSS.sub_pico.20230517_115439.proc.nwb
Normalized and Original PSTH (1183, 15, 30, 192) (1183, 15, 30, 192)
Loading files from day 2/2
Within: using normalizer file norm_FOSS.sub_pico.20230518_103908.proc.nwb
Normalized and Original PSTH (1183, 15, 30, 192) (1183, 15, 30, 192)
Saving combined NWB Files.
Combined file saved.
Train file saved.
Test file saved

### Step 3: Validate the experiment nwb files

In [3]:
experiment_file_paths = glob.glob(os.path.join(root_dir, '[exp]*', '*'))
for experiment_path in experiment_file_paths:
    if os.path.basename(experiment_path).startswith('exp'): 
        path = os.path.join(experiment_path, f"{os.path.basename(experiment_path)}_combined.nwb")
        if os.path.isfile(path):
            try:
                    io = NWBHDF5IO(path, "r") 
                    nwbfile = io.read()
                    display(nwbfile)
                    io.close()
                    break
            except: print(f'This File can not be opened: {os.path.basename(experiment_path)}')
            
        else: print(f'No combined nwb found in: {os.path.basename(experiment_path)}')



No combined nwb found in: exp_muri1320.sub_pico


root pynwb.file.NWBFile at 0x140371379827088
Fields:
  devices: {
    DET36A2 Biased Si Detector <class 'pynwb.device.Device'>,
    Electrode <class 'pynwb.device.Device'>,
    LG UltraGear <class 'pynwb.device.Device'>,
    RecordingSystem <class 'pynwb.device.Device'>,
    Ripple Neuro <class 'pynwb.device.Device'>,
    Software Used <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    group_A <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_B <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_C <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_D <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_E <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_F <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_G <class 'pynwb.ecephys.ElectrodeGroup'>,
    group_H <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: Task: Rapid serial visual presentation (RSVP).
  experimenter: ['Goulding, Sarah']
  file_create_date: [datetime.datetime(2023, 12, 6, 16, 3, 49, 71689, tzinfo=tzoffset(None, -18000))]
  identifier: a0cdc718-58a3-4566-a2e0-f0d8216fd580
  institution: McGovern Institute for Brain Research
  keywords: <StrDataset for HDF5 dataset "keywords": shape (4,), type "|O">
  lab: DiCarlo
  scratch: {
    QualityCheckedPSTH_20220930_153358 <class 'pynwb.core.ScratchData'>,
    QualityElectrodesMasks <class 'pynwb.core.ScratchData'>
  }
  session_description: Integrated NWB file for the experiment, featuring PSTH data verified for quality from each session.
  session_id: exp_ko_context_size
  session_start_time: 2022-09-30 15:33:58-04:00
  stimulus_template: {
    StimulusSet <class 'pynwb.base.Images'>
  }
  subject: subject pynwb.file.Subject at 0x140365871689616
Fields:
  age__reference: birth
  date_of_birth: 2014-06-22 00:00:00-04:00
  description: monkey
  sex: M
  species: Macaca mulatta
  subject_id: pico

  surgery: 3x Utah Array Implant + Headpost
  timestamps_reference_time: 2022-09-30 15:33:58-04:00